In [ ]:
# RUN THIS CELL FOR FORMAT
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

<a id="contents"></a>

## Table of Contents

- [**1. Introduction**](#introduction)

> - [**1.1 Derivatives and Differentiation Methods**](#introduction1)
> - [**1.2 Differentiation Methods and Motivations**](#introduction2)
> - [**1.3 Automatic Differentiation**](#introduction3)

- [**2. Background**](#background)

> - [**2.1 Chain Rule**](#background1)

> - [**2.2 Elementary Functions**](#background2)

> - [**2.3 Forward Mode**](#background3)

> - [**2.4 Reverse Mode**](#background4)

> - [**2.5 Dual Number**](#background5)

- [**3. Usage**](#usage)

> - [**3.1 Installation**](#usage1)

> - [**3.2 How to use**](#usage2)

- [**4. Software Organization**](#softwareorganization)

- [**5. Implementation**](#implementation)

- [**6. Reference**](#reference)


<a id="introduction"></a>

## Introduction

[Return to contents](#contents)

[package name], [package description], is a PyPi-distributed package that executes forward-mode and reverse-mode of automatic differentiation, enabling users to solve functional derivatives with high computational efficiency and machine precision.

<a id="introduction1"></a>

###1.1 Derivatives

[Return to contents](#contents)

The derivative measures the sensitivity of the function value with respect to changes in its arguments. In one dimensional space, let $I \subseteq \mathbb{R}$ be an interval and let $x_0 \in I$. If exists, the derivative of the real-valued, single variable function $f \colon I → \mathbb{R}$ at $x_0$ is defined as,

$$f'(x_0) =\lim_{h→0} \frac{f(x_0+h)-f(x_0)}{h}$$

In multiple dimensions, for a vector-valued function of several variables, such as the mapping $f \colon \mathbb{R}^{m} → \mathbb{R}^{n}$, the partial derivative of the $i$th vector component of function output with respect to the $j$th input can be written as,

$$\frac{\partial}{\partial{x_j}}f_i(\textbf{x}) = \lim_{h→0}\frac{f_i(x_1, x_2, ..., x_{j-1}, x_{j}+h, x_{j+1},...,x_{m})-f_i(x_1, x_2, ..., x_{j-1}, x_{j}, x_{j+1},...,x_{m})}{h}$$


In this case, the Jacobian matrix consisting of all the first-order partial derivatives of the function is often considered. The Jacobian $J(\textbf{x}^{(k)})$ of $f(\textbf{x})$ evaluated at $\textbf{x}^{(k)}$ will be a $n \times m$ matrix with elements 

$$J(\textbf{x}^{(k)})_{i,j} = \frac{\partial{f_i}}{\partial{x_j}}\Bigr\rvert_{\textbf{x} = {x}^{(k)}}$$

<a id="introduction2"></a>

###1.2 Differentiation Methods and Motivations

[Return to contents](#contents)


Differentiation is one of the most fundamental operations in science with a wide range of applications, including optimization, linearization, and root-finding problems. Several approaches to carry out the task have been proposed and gained great ubiquity:
 * Symbolic calculus methods accept an expression of a function and return the derivative of the given formula with respect to a specified variable. Despite offering analytical solutions at one's disposal, which has benefits in and of itself, symbolic mathematics programs usually have a high computation cost and may not always be applicable especially in cases of discontinuity, high complexity, and non-smoothness while finding the exact form of the derivative as a function. 
 * Numerical differentiation, such as the three-point forward difference formula, employs finite differences approximation to find rate of change of function value at a given point in the domain. Due to the nature of the approximation, it suffers from inaccuracy and instability.
 * In comparison, the automatic differentiation algorithm has the advantage of attaining both low computational costs and machine precision, overcoming the deficiencies of the two other methods introduced above. Its superior performance lies in its intricate design and implementation, which we will cover in the next section.

<a id="introduction3"></a>

###1.3 Automatic Differentiation

[Return to contents](#contents)

As mentioned above, Automatic Differentiation is a compotent tool for computing the Jacobian with reasonable computational costs and optimal accurcay, and thus making great contribution to progress made in fluid dynamics, aerospace, and so on. For instance, the back-propagation algorithm, a special case of reverse mode AD applied to scaler functions, is utilized to update node weights of neural network in machine learning. Here, [package name] provides a robust, readily accessible implementation of the algorithm.

<a id="background"></a>

##2. background
AD calculates derivatives with the same accuracy as symbolic differentiation, but operates directly on the program of interest to obtain numerical values. The fundamental principle of AD is to decompose complex functions into elementary functions with known derivatives that form evaluation traces, and apply the chain rule to compose the derivatives of each step. Hence we first introduce chain rule and elementary function in order to understand AD.

[Return to contents](#contents)

<a id="background1"></a>

###2.1 Chain Rule



[Return to contents](#contents)

The central idea of Automatic Differentiation is the decomposition of complicated functional relations and the piecewise determination of derivatives based on the generalized Chain Rule.  

To recall, according to the Chain Rule, for scalar-valued function of one real variable $f(x)$ and $g(x)$,

$$\frac{\text{d}}{\text{d}x} (f(g(x))) = f'(g(x))g'(x)$$

When extended to real-valued functions of multiple variables in high dimensional coordinates $f\colon \mathbb{R}^n → \mathbb{R}$, the Chain Rule states that,

$$\nabla_{\textbf{x}} f = \sum_{i=1}^{n} \frac{\partial{f}}{\partial{y_i}} \nabla y_i(\textbf{x})$$

where $\textbf{x} = [x_1,x_2,...,x_m] \in \mathbb{R}^m$ vector of independent coordinates, $y(\textbf{x}) = [y_1(\textbf{x}), y_2(\textbf{x}), ...,y_n(\textbf{x})] \in \mathbb{R}^n$ n-dimenstional input vector of $f$, and $\nabla y_i(\textbf{x}) = [\frac{\partial{y_i}}{\partial{x_1}}, \frac{\partial{y_i}}{\partial{x_2}}, ...,\frac{\partial{y_i}}{\partial{x_m}}]{ᵀ}$ 


In the most generalized form, the output of $f$ would be a vector instead of a scaler, in which case the derivative of each component $f_k$ can be following the Chain Rule. 









<a id="background2"></a>

###2.2 Elementary Functions

[Return to contents](#contents)

A complex function formula is made up of basic unitary and binary operations, a concept known as the partial ordering of the operations associated with the function $f$. Using these operations as delimiters, a function can be broken down into a handful parallel and enclosing elementary functions. 

Specifically, an elementary function is a function of a single variable that is defined as taking sums, products, roots and compositions of finitely many polynomial, rational, trigonometric, hyperbolic, and exponential functions, including possibly their inverse functions.$^1$Some common examples include,

* Constant functions: 2, $\pi$,$e$
* Rational powers of x: $x$,$x^3$,$\sqrt{x}$
* Exponential and logrithmic functions: $e^{x}$, $\ln{x}$, $10^{x}$,$\log_2{x}$
* Trignometric functions: $\sin(x)$

With evaluate intermediate results
intermediate steps

<a id="background3"></a>

###2.3 Forward Mode

Forward Mode is prefered when computing Jacobians where n is much lesser than m, i.e. the dimension of inputs significantly smaller than the dimension of outputs.
$$ f: \mathbb{R}^{n} → \mathbb{R}^{m}$$

In forward mode, we compute the inner product of $$\gradient f * p$$
where $\gradient f$ refers to the Jacobian and p refers to the seed vector.
####2.3.1 Jacobian
####2.3.2 Seed Vector

Here is an exmaple

add computational graph, trace table
[Return to contents](#contents)

<a id="background4"></a>

###2.4 Reverse Mode

The reverse mode is ideal when the dimensionality of the input is significantly larger than the dimensionality of the output. Instead of propagating the derivative forward as the forward mode does, the reverse mode propagates the derivative backward from the output. The reverse mode consists of two parts: forward pass and reverse pass. 
During the forward pass, we evaluate the intermediate variables $v_{i}$, 

####2.4.1 Forward Pass
During the forward pass, we evaluate the intermediate variables $v_{i}$ and store the partial derivatives of child nodes with respect to their parent node in memory$$\frac{\partial{v_{j}}}{\partial{v_{i}}}$$

####2.4.2 Reverse Pass
In reverse mode, we compute the paritial derivates of the output with respect to the intermediate variables known as adjoints iteratively. The adjoint is denoted as $v_{i}$, where
$$ \bar{v_{i}} = \frac{\partial{f}}{\partial{v_{i}}} = \sum_{j: child of i} \frac{\partial{f}}{\partial{v_{j}}}\frac{\partial{v_{j}}}{\partial{v_{i}}}$$

For each specific node, we first apply chain rule to multiple the adjoint of child by the partial derivate of the child with respect to $v_{i}$ and take the sum of these products. In other words, $v_{i}$'s contribution to the output is defined by its children's contribution to the output as well as its affect on children. In this case, we obtain partial derivate of each input. 

####2.4.3 Comparison to Forward Mode
In contrast to the forward mode, the reverse mode of AD does not involve directional derivatives. Instead, it calculates and stores the partial derivatives. The reverse mode can only run after the forward pass is completed and it is memeory-intensive

[Return to contents](#contents)

<a id="background5"></a>

###2.5 Dual Number

A dual number consists of a real part and a dual part, with the following format $$z=a+bϵ$$ where a represents the real part $f$, b represents the dual part $f\prime$, and $\epsilon$ is a nonzero and nonreal number which satisfies $\epsilon^2=0$.

Noticing the format of dual numbers is very similar to complex numbers. The key difference here is that for dual numbers $\epsilon^2=0$ whereas for complex number $i^2=-1$.

Dual numbers are extremely useful when calculating function values and function derivatives in AD setups. For instance, say we have $$f(x) = x^2$$ and would like to calculate $f(x)$ and $f\prime(x)$. We first convert x into a dual number as $x=a+bϵ$. Then $$f(x) = (a+bϵ)^2 = a^2 + 2abϵ + b^2ϵ^2 = a^2 + abϵ $$ where $f(x) = a^2$ and $f\prime(x) = 2abϵ$. 

[Return to contents](#contents)

<a id="usage"></a>

##3. Usage

[Return to contents](#contents)

[package name] is available through the Python Package Index (PyPI). Refer to the PyPI webpage for more project description.

<a id="usage1"></a>

###3.1 Installation

[Return to contents](#contents)

To install [package name] with pip, run the following command:

    pip install [package name]

<a id="usage2"></a>

###3.2 How to use

[Return to contents](#contents)

Aftering successful installation, the user can import [package name] and perform AD computation.

The package provides the following APIs:


*   Instanitiate AD objects, with forward/reverse mode specified
*   Compute the value of the function
*   Compute the gradient/Jacobian of the function
*   Compute the directional derivative of the function with seed vector p specified

The sample code illustrates how to use the package.

<a id="softwareorganization"></a>

##4. Software Organization

[Return to contents](#contents)

* Directory Structure

```
├── __init__.py
├── Docs
│   ├── README.md
│   ├── milestone1.md
│   │   ...
└───AD4fun
│   ├── README.md
│   ├── forward_mode_class.py
│   ├── reverse_mode_class.py
│   ├── elementary_functions.py
│   ├── dual_number_class.py
│   ├── core_data_classes.py
│   │   ...
│
└───Test toolbox
│   ├── README.md
│   ├── core_data_initialization_test.py
│   ├── forward_mode_test.py
│   ├── reverse_mode_test.py
│   ├── elementary_function_test.py
│   ├── class_dual_number_test.py
│   ├── formula_conversion_test.py
│   │   ...

```

**Packaging & Distribution**

We build and package our software using the the standard packaging tool. [setuptools](https://packaging.python.org/key_projects/#setuptools)

Our code follows PEP8 standard.

Our package would be distributed through [Python Package Index (PyPi)](https://pypi.org/)

<br>

**Modules to Include**

 math: performing elementary functions and mathematic algebric operations

 numpy: provide multi-dimensional array and matrix computation support

<br>

 **Test Toolbox Design**

The functionality tests will be included in the test toolbox directory in the root directory. We plan to use TravisCI to test coverage and integration.


<a id="implementation"></a>

##5. Implementation

[Return to contents](#contents)

#### Classes

####5.1 Core Data structures

As per our current design, we plan to use the following core data structures - all computations and functions would be implemented around those central classes

1. `Formula` objects

This class will have many subclasses each handling one kind of mathematical expressions such as sine, cosine, natural logrithm etc. This class provides analytical differentiation and numeric evaluation interfaces which would be implemented in each subclass. Any mathematical expression would be converted into `Formula` objects. For instance, f = sin(x)**2 would be a power formula object within which contains a sine formula object - that sine object in turn contains a variable formula object.


2. `Variable` objects

`Variable` class is a special child class of formula class that represents the independent variables of a mathematical functions. For instance x,y,z  in f(x,y,z) = x + y + z
These objects are against which mathematical functions are differentiated. 

3. `Variable-to-Value map` dictionary

We use a python dictionary to store the hash map from each variable to its user assigned values. For instance {x:3} stores the information that x=3

####5.2 Implemented Classes

* Forward mode: This class will take in a list of `Formula` objects, call each Formula's own differention method, plug in differentiation result with user defined dual values for each gradient result to obtain dual number result for each function and return a list of results.

* Reverse mode: This class will compute derivatives for each input `Formula` object through reverse mode and return a list of results.

* Elementary function: This series of classes will overload any elementary functions such as addition, subtraction, multiplication, sine function etc. such that any computation involving a Formula object would be converted into a Formula, too. For instance f=3+x would result in f becoming a `Addition Formula`object

* Dual Number Class: This class will convert any real number input and return it as a dual number,
this saves computational power in the final plug-in-value phase of evaluating differentiation result

(2) Reverse mode

The trace function will also be able to calculate derivatives in reverse mode by specifying the mode parameters. Take the example below as a demo

The table below shows some examples of elementary functions and their respective derivatives:

| Elementary Function    | Derivative              |
| :--------------------: | :----------------------:|
| $x^3$                  | $3 x^{2}$               | 
| $e^x$                  | $e^x$                   |
| $\sin(x)$              | $\cos(x)$               |
| $\ln(x)$               | $\frac{1}{x}$           |

<a id="reference"></a>

##6. Reference

[Return to contents](#contents)

In [ ]:

# user defined function
f = lambda x: x - np.exp(-2.0 * np.sin(4.0 * x) * np.sin(4.0 * x))

# call the trace function in undefined, and provide input x = 2
nd.trace(f, x = 2, mode = 'reverse')

# the function will return the 1st derivative when x=2.
>>> taking derivative...
>>> 0.674811


SyntaxError: ignored